<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Fine-Tuning BERT</center></span>

<center>Transfer Learning en Transformers</center>

##   <span style="color:blue">Profesores</span>

### Coordinador


- Campo Elías Pardo Turriago, cepardot@unal.edu.co 

### Conferencistas


- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 
- Oleg Jarma, Estadístico, ojarmam@unal.edu.co 

## <span style="color:blue">Asesora Medios y Marketing digital</span>

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 



## <span style="color:blue">Asistentes</span>



- Nayibe Yesenia Arias, naariasc@unal.edu.co
- Venus Celeste Puertas, vpuertasg@unal.edu.co 

## <span style="color:blue">Referencias</span>

1. [How to Train BERT](https://towardsdatascience.com/how-to-train-bert-aaad00533168), James Briggs, Towards Data Science
1. [Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training), Tutoriales de HuggingFace
1. [The untold story of GPT-3 is the transformation of OpenAI](https://bdtechtalks.com/2020/08/17/openai-gpt-3-commercial-ai/), Techtalks
1. [OpenAI’s massive GPT-3 model is impressive, but size isn’t everything](https://venturebeat.com/2020/06/01/ai-machine-learning-openai-gpt-3-size-isnt-everything/), VentureBeat
1. [Aprendizaje Profundo-Diplomado](https://github.com/AprendizajeProfundo/Diplomado)
1. [Aprendizaje Profundo-PLN](https://github.com/AprendizajeProfundo/PLN)
1. [Ashish Vaswani et al.,   Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf), diciembre 2017.
1. [Dennis Rothman, Transformers for Natural Language processing](http://libgen.rs/search.php?req=Transformers+for+Natural+Language+processing&open=0&res=25&view=simple&phrase=1&column=def), enero 2021.
1.[ Varios,  Dive into deep learning](https://d2l.ai/), enero 2021

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Cargar Modelo Base](#Cargar-Modelo-Base)
* [Cargar Datos](#Cargar-Datos)
* [Dividir Datos para la tarea específica (NSP)](#Dividir-Datos-para-la-tarea-específica-(NSP))
* [Construir Dataset que espera el modelo](#Construir-Dataset-que-espera-el-modelo)
* [Tokenizar Frases](#Tokenizar-Frases)
* [Construir etiquetas para la tarea MLM (Masking)](#Construir-etiquetas-para-la-tarea-MLM-(Masking))
* [Construir Dataset y DataLoader](#Construir-Dataset-y-DataLoader)
* [Fine-Tuning BERT](#Fine-Tuning-BERT)
* [Fine-Tune para otras tareas (HuggingFace)](#Fine-Tune-para-otras-tareas-(HuggingFace))
* [Fine-Tune SentenceBert](#Fine-Tune-SentenceBert)
* [Pytorch Lightning](#Pytorch-Lightning)

## <span style="color:blue">Introducción</span>

El poder de los transformadores se deriva de la práctica común de que los modelos de transformadores sean entrenados previamente con un estándar muy alto por parte de grandes empresas como Google y OpenAI.

Ahora, cuando digo preentrenado con un estándar muy alto, los costos de entrenamiento estimados de GPT-3 de OpenAI oscilan entre 4.6 y 12 millones de dólares.

## <span style="color:blue">Cargar Modelo Base</span>

Comencemos llamando al modelo base junto con su tokenizador.

Observe que para entrenar BERT, debemos llamar la clase *BertForPreTraining* y no *BertModel*, pues el primero se utiliza para entrenar BERT de la manera clásica (NSP y MLM).

Por ejemplo, BETO (BERT en español), fue entrenado utilizando ésta técnica sobre un conjunto masivo de datos en español.

Para mayor aclaración sobre el asunto, se puede consultar [BertModel or BertForPreTraining](https://stackoverflow.com/questions/66596142/bertmodel-or-bertforpretraining#:~:text=BertForPreTraining%20is%20used%20to%20train,build%20on%20top%20of%20it.), en StackOverFlow.

In [ ]:
from transformers import BertTokenizer, BertForPreTraining
import torch

# Tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Modelo BERT para entrenar NSP y MLM
model = BertForPreTraining.from_pretrained('bert-base-uncased')

## <span style="color:blue">Cargar Datos</span>

Utilicemos un conjunto de datos sobre el que queramos entrenar BERT. Los datos utlizados en esta lección pueden ser encontrados en [jamescalam
/
transformers](https://github.com/jamescalam/transformers/blob/main/data/text/meditations/clean.txt).

Estos textos tratan sobre 507 pasajes de meditaciones personales de James Briggs.

Cada pasaje puede tener una o más frases.

In [ ]:
with open('../Datos/clean.txt', 'r') as fp:
    # Leer líneas del texto
    text = fp.read().split('\n')
    
# Mostrar ejemplo
text[:3]

## <span style="color:blue">Dividir Datos para la tarea específica (NSP)</span>

Ahora que hemos cargado los datos, debemos dividir el conjunto de datos para poder etiquetar las frases que siguen las unas a las otras.

Esto se conoce como la tarea NSP (Next Sentence Prediction).

In [4]:
# Separar frases de cada uno de los párrafos extraídos
bag = [item for sentence in text for item in sentence.split('.') if item != '']
# Tamaño de la bolsa de frases
bag_size = len(bag)

print(bag_size)
text[14]

'From Maximus I learned self-government, and not to be led aside by anything; and cheerfulness in all circumstances, as well as in illness; and a just admixture in the moral character of sweetness and dignity, and to do what was set before me without complaining. I observed that everybody believed that he thought as he spoke, and that in all that he did he never had any bad intention; and he never showed amazement and surprise, and was never in a hurry, and never put off doing a thing, nor was perplexed nor dejected, nor did he ever laugh to disguise his vexation, nor, on the other hand, was he ever passionate or suspicious. He was accustomed to do acts of beneficence, and was ready to forgive, and was free from all falsehood; and he presented the appearance of a man who could not be diverted from right rather than of a man who had been improved. I observed, too, that no man could ever think that he was despised by Maximus, or ever venture to think himself a better man. He had also the

In [ ]:
# Mostrar ejemplo de bolsa de frases
bag[14:19]

## <span style="color:blue">Construir Dataset que espera el modelo</span>

Ahora seleccionaremos 50/50 frases al azar que siguen unas de las otras y las marcaremos.

Aquí, 0 representa la siguiente frases y 1 que no es la siguiente frase.

In [6]:
import random

sentence_a = []
sentence_b = []
label = []

# Seleccionar párrafo
for paragraph in text:
    # Separar frases en el párrafo
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != ''
    ]
    # Contar frases
    num_sentences = len(sentences)
    # Debe haber al menos una frase
    if num_sentences > 1:
        # Seleccionar índice al azar
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        # Lanzar moneda
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            # Anexar pares correspondientes
            sentence_b.append(sentences[start+1])
            # Crear etiqueta de IsNextSentence
            label.append(0)
        else:
            # De otra manera, seleccionar de la bolsa de frases
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            # Anexar parejas no correspondientes
            sentence_b.append(bag[index])
            # Crear etiqueta de NotNextSentence
            label.append(1)

Veamos un ejemplo de cómo quedó nuestro conjunto de datos.

In [7]:
for i in range(3):
    # Seleccionar etiqueta
    print(label[i])
    # Seleccionar primer par de frases
    print(sentence_a[i] + '\n---')
    # Seleccionar segundo par de frases
    print(sentence_b[i] + '\n')

0
 He was accustomed to do acts of beneficence, and was ready to forgive, and was free from all falsehood; and he presented the appearance of a man who could not be diverted from right rather than of a man who had been improved
---
 I observed, too, that no man could ever think that he was despised by Maximus, or ever venture to think himself a better man

1
 Further, he was not fond of change nor unsteady, but he loved to stay in the same places, and to employ himself about the same things; and after his paroxysms of headache he came immediately fresh and vigorous to his usual occupations
---
 Show him where his error is

1
 Further, I owe it to the gods that I was not hurried into any offence against any of them, though I had a disposition which, if opportunity had offered, might have led me to do something of this kind; but, through their favour, there never was such a concurrence of circumstances as put me to the trial
---
 For as every soul is unwillingly deprived of the truth, so

## <span style="color:blue">Tokenizar Frases</span>

Como siguiente paso, debemos tokenizar las frases de nuestro conjunto creado,teniendo en cuenta los argumentos del tokenizador creados para esta tarea (Observe como se pasan los textos).

In [8]:
# Máxima longitud de tokens -> 512
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt',max_length=512, truncation=True, padding='max_length')

Como podemos observar, el output del tokenizador es u diccionario que contiene los items tokenizados y listo para pasar por el modelo.

In [17]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [18]:
inputs

{'input_ids': tensor([[  101,  2002,  2001,  ...,     0,     0,     0],
        [  101,  2582,  1010,  ...,     0,     0,     0],
        [  101,  2582,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Note que en el DataFrame los tipos de tokens cambian (son uno), para indicar que las frases son pares, y luego los ceros finales indican el padding hasta 52 tokens.

In [27]:
import pandas as pd
pd.set_option('display.max_columns',512)
pd.DataFrame(inputs.token_type_ids)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
313,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
314,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ahora debemos convertir las etiquetas en un tensor de pytorch para poder procesarlos después:

In [28]:
# Convertir etiquetas en tensor dentro del diccionario de los inputs
inputs['next_sentence_label'] = torch.LongTensor([label]).T

In [30]:
#Ejemplo
print(inputs.next_sentence_label[:10].T)

tensor([[0, 1, 1, 0, 1, 1, 1, 0, 1, 1]])


## <span style="color:blue">Construir etiquetas para la tarea MLM (Masked Language Model)</span>

Ya tenemos listo el modelo para la tarea NSP.

Ahora debemos preparar nuestros datos para la tarea de Masking.

Esto es, quitar palabras al azar de las frases y tratar de recuperarlas con el modelo.

Para este caso, como es un proceso de clasificación, los tokens serán nuestra etiqueta dentro del vocabulario del modelo.

In [31]:
# labels para la tarea MLM
inputs['labels'] = inputs.input_ids.detach().clone()

In [32]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

In [33]:
# crear una matriz aleatoria de floats con dimensiones iguales al tensor input_ids
rand = torch.rand(inputs.input_ids.shape)
# Crear arreglo de máscara (15% del tiempo) y quitar tokens especiales del proceso
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [34]:
selection = []

# Tomar los índices de valores True en cada vector
for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [35]:
# Índices seleccionados
selection[:2]

[[2, 6, 9, 12, 17, 22, 35, 36, 43, 53, 64],
 [11, 21, 32, 33, 34, 41, 44, 45, 57]]

In [36]:
# Colocar índice especial para la máscara
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [37]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

In [38]:
# Tensor de inputs con máscara
inputs.input_ids

tensor([[  101,  2002,   103,  ...,     0,     0,     0],
        [  101,  2582,  1010,  ...,     0,     0,     0],
        [  101,   103,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]])

## <span style="color:blue">Construir Dataset y DataLoader</span>

Para acelerar el proceso de entrenamiento y poder hacerlo por batches, creamos un dataset y un dataloader respectivamente para el modelo.

In [ ]:
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # Obtener codificación
        self.encodings = encodings
        # Obtener items
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    # Obtener longitud del dataset
    def __len__(self):
        return len(self.encodings.input_ids)

In [40]:
# instanciar dataset
dataset = OurDataset(inputs)

In [41]:
# Generar DataLoader (batches de 16)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

## <span style="color:blue">Fine-Tuning BERT</span>

Ya que tenemos nuestros datos listos, ahora podemos entrenar el modelo y hacer nuestro fine-tuning.

EL loop de training va como sigue:

In [47]:
# Barra de progreso
from tqdm import tqdm
# Cargar Optimizadores
from torch.optim import Adam,AdamW
# Otpimizador recoge información sobre los parámetro del modelo BERT
optim = AdamW(model.parameters())

# Si tienes GPU, debes modificar esta línea
device='cpu'

# Dos epochs
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # Inicializar gradiente (sin acumulación)
        optim.zero_grad()
        # Colocar batches de información en el dispositivo indicado
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        labels = batch['labels'].to(device)
        # Red forward que usa NSP y MLM
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        # Calcular pérdida
        loss = outputs.loss
        # Calcular gradiente
        loss.backward()
        # actualizar parámetros
        optim.step()
        # Mostrar información importante
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|                                                                                                                                                   | 0/20 [00:00<?, ?it/s]/var/folders/d_/hfs18jf54371qfmf54zq289c0000gn/T/ipykernel_76545/541448751.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [15:06<00:00, 45.31s/it, loss=2.06]


Observemos cómo la pérdida baja de un epoch a otro.

También, note lo costoso comoputacionalmente que es el proceso, pero comparado con inicializar el modelo desde cero, es un costo que podemos aceptar.

## <span style="color:blue">Fine-Tune para otras tareas (HuggingFace)</span>

Ahora exploremos tareas **Downstream**, es decir, utilicemos BERT para otra tarea que no sea NSP o MLM.

En esta sección construiremos un clasificador multi-clase para análisis de sentimientos.

Para este caso, utilizaremos un dataset de HuggingFace.

Si quiere replicar este proceso tal cual como se indica en este ejemplo, deberá construir un dataset de HuggingFace.

De otra manera, puede seguir el mismo proceso hecho con anterioridad (DataLoader) para crear los datos.

In [49]:
from datasets import load_dataset

# usaremos el dataset yelp_review_full
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /Users/moury/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

La filosofía de HuggingFace: *Make things simple and easy.*

Es por eso que esta plataforma cuenta con un AutoTokenizador, que se encarga de tomar el tokenizador adecuado para el modelo especificado.

Otra cosa interesante de los datasets de HF, es que permite el uso de la función `map` para procesar alto volúmenes de información en forma de batches.

In [50]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

Como este conjunto es bastante grande, seleccionaremos sólo un conjunto de 1000 reviews.

In [55]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

También, usaremos AutoModelFor**Task**. Observe lo fácil que es construir el clasificador con un número de etiquetas especificado (5 en este caso).

In [51]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Dentro de los argumentos de entrenamiento, podemos especificar, por ejemplo, la carpeta dónde quedará nuestro checkpoint, etc...

In [56]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

También podemos traer métricas de la clase dataset

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

Generamos las predicciones y calculamos el accuracy del modelo:

In [59]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Además podemos elegir la forma de evaluación del modelo, que en este caso hemos elegido como "epoch" (3 por defecto).

In [60]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Y ahora viene lo más fabuloso de HuggingFace:

Su API Trainer para hacer los procesos más flexibles para investigación:

Observe lo sencillo que es generar el modelo.

In [61]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Y lo fácil que es entrenarlo...

In [ ]:
trainer.train()

/Users/moury/miniforge3/envs/numpy_test/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.219129,0.498000
2,No log,1.122104,0.506000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Ahora bien, para una epoch y sin GPUs, el modelo tarda approx 2 horas/epoch. Mejor vaya por un tinto y vuelva más tarde.

**Enhorabuena,** usted ha hecho su primer clasificador utilizando un modelo BERT.

Llámelo BERT-5CR (BERT 5 Class Review).

## <span style="color:blue">Fine-Tune SentenceBert</span>

También es posible hacer fine-tuning de modelos SentenceBert, como se indica a continuación:

In [13]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

In [14]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [15]:
sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

  0%|          | 0.00/392k [00:00<?, ?B/s]

In [16]:
# Read the dataset
model_name = 'nli-distilroberta-base-v2'
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [17]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")

2022-02-28 15:06:28 - Load pretrained SentenceTransformer: nli-distilroberta-base-v2


Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

2022-02-28 15:07:04 - Use pytorch device: cpu
2022-02-28 15:07:04 - Read STSbenchmark train dataset


In [18]:
train_samples = []
dev_samples = []
test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

        if row['split'] == 'dev':
            dev_samples.append(inp_example)
        elif row['split'] == 'test':
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [35]:
type(train_samples[0])

sentence_transformers.readers.InputExample.InputExample

In [26]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

In [27]:
logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

2022-02-28 15:12:12 - Read STSbenchmark dev dataset


In [29]:
# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2022-02-28 15:14:15 - Warmup-steps: 144


In [30]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

2022-02-28 15:42:08 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2022-02-28 15:42:46 - Cosine-Similarity :	Pearson: 0.8866	Spearman: 0.8866
2022-02-28 15:42:46 - Manhattan-Distance:	Pearson: 0.8566	Spearman: 0.8599
2022-02-28 15:42:46 - Euclidean-Distance:	Pearson: 0.8578	Spearman: 0.8616
2022-02-28 15:42:46 - Dot-Product-Similarity:	Pearson: 0.8350	Spearman: 0.8354
2022-02-28 15:42:46 - Save model to output/training_stsbenchmark_continue_training-nli-distilroberta-base-v2-2022-02-28_15-04-13


Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

2022-02-28 15:57:40 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2022-02-28 15:58:17 - Cosine-Similarity :	Pearson: 0.8903	Spearman: 0.8902
2022-02-28 15:58:17 - Manhattan-Distance:	Pearson: 0.8631	Spearman: 0.8663
2022-02-28 15:58:17 - Euclidean-Distance:	Pearson: 0.8647	Spearman: 0.8685
2022-02-28 15:58:17 - Dot-Product-Similarity:	Pearson: 0.8444	Spearman: 0.8452
2022-02-28 15:58:17 - Save model to output/training_stsbenchmark_continue_training-nli-distilroberta-base-v2-2022-02-28_15-04-13


Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

2022-02-28 16:13:52 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2022-02-28 16:14:38 - Cosine-Similarity :	Pearson: 0.8890	Spearman: 0.8897
2022-02-28 16:14:38 - Manhattan-Distance:	Pearson: 0.8718	Spearman: 0.8747
2022-02-28 16:14:38 - Euclidean-Distance:	Pearson: 0.8736	Spearman: 0.8772
2022-02-28 16:14:38 - Dot-Product-Similarity:	Pearson: 0.8527	Spearman: 0.8530


Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

2022-02-28 16:29:25 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2022-02-28 16:30:05 - Cosine-Similarity :	Pearson: 0.8894	Spearman: 0.8896
2022-02-28 16:30:05 - Manhattan-Distance:	Pearson: 0.8712	Spearman: 0.8751
2022-02-28 16:30:05 - Euclidean-Distance:	Pearson: 0.8727	Spearman: 0.8774
2022-02-28 16:30:05 - Dot-Product-Similarity:	Pearson: 0.8536	Spearman: 0.8540


## <span style="color:blue">Pytorch Lightning</span>

Finalmente, si desea hacer modelos en muchos menos pasos, con autoidentificación de dipositivos disponibles, le presentamos **Pytorch Lightning**, la forma más fácil de hacer modelos en el tiempo actual sin sufrir en el intento en Pytorch.

El paradigma de PL es que uno sólo debe sobreescribir las clases que se quieren configurar. El resto lo maneja por debajo PL.

**¡Bienvenido a la nueva era!**

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from transformers import BertModel
from torch.optim import Adam,AdamW

class Clasificador_BERT(pl.LightningModule):
    def __init__(self,dropout=0.2):
        super().__init__()
        self.bert = BertModel.from_pretrained('hiiamsid/sentence_similarity_spanish_es')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()
        self.loss = F.cross_entropy
        
    def forward(self,input_id,mask):
        # in lightning, forward defines the prediction/inference actions
        _, pooled_output = self.bert(input_ids = input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x_id,x,y = batch
        # USE MODEL
        _,z = self.bert(input_ids = x['input_ids'].squeeze(1),attention_mask = x['attention_mask'],return_dict=False)
        z   = self.dropout(z)
        z   = self.linear(z)
        output = self.relu(z)
        
        loss = self.loss(output, y)
        # Logging to TensorBoard by default
        self.log("train_loss", loss,prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-6)
        return optimizer